In [1]:
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.pyplot as plt
import string
import re
import numpy as np

In [2]:
# use this do download stopwords if you haven't done this before
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# loading in dataframe, using special encoder parameter to save it correctly
df = pd.read_csv('twitter_data.csv',encoding= 'unicode_escape')

In [4]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [5]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [6]:
# renaming the columns
df = df.rename(columns={"tweet_text": "tweet", "emotion_in_tweet_is_directed_at": "subject", "is_there_an_emotion_directed_at_a_brand_or_product": "emotion"})

In [7]:
df.head()

,tweet,subject,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [8]:
df['subject'].value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: subject, dtype: int64

In [9]:
df['tweet'] = df['tweet'].str.lower()

In [10]:
df['subject'].isna().sum()

5802

In [11]:
df['emotion'].isna().sum()

0

In [12]:
df['emotion'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: emotion, dtype: int64

In [13]:
df.loc[df['emotion'] == "I can't tell"]['tweet'][237]

'just what america needs. rt @mention google to launch major new social network called circles, possibly today {link} #sxsw'

In [14]:
df.loc[df['emotion'] == "I can't tell"]['subject'].isna().sum()

147

In [15]:
df['tweet'].isna().sum()

1

In [16]:
df_binary = df.loc[(df['emotion'] == 'Positive emotion' ) | (df['emotion'] == 'Negative emotion' )]

In [17]:
df.loc[df['emotion'] == 'No emotion toward brand or product']['tweet'][9091]

'some verizon iphone customers complained their time fell back an hour this weekend.  of course they were the new yorkers who attended #sxsw.'

In [18]:
df_binary['subject'].isna().sum()

357

In [19]:
len(df_binary)

3548

In [20]:
df_binary.loc[df['subject'].isna()]['tweet']

46      hand-held û÷hoboûª: drafthouse launches û÷h...
64      again? rt @mention line at the apple store is ...
68      boooo! rt @mention flipboard is developing an ...
103     know that &quot;dataviz&quot; translates to &q...
112     spark for #android is up for a #teamandroid aw...
                              ...                        
9011    apparently the line to get an ipad at the #sxs...
9043    hey is anyone doing #sxsw signing up for the g...
9049    @mention you can buy my used ipad and i'll pic...
9052    @mention you could buy a new ipad 2 tmrw at th...
9054    guys, if you ever plan on attending #sxsw, you...
Name: tweet, Length: 357, dtype: object

In [21]:
df_binary.loc[df['subject'] == 'Apple']['tweet']

9       counting down the days to #sxsw plus strong ca...
40      @mention  - great weather to greet you for #sx...
47      hooray rt ûï@mention apple is opening a pop-u...
49      wooooo!!! ûï@mention apple store downtown aus...
62      #omfg! rt @mention heard about apple's pop-up ...
                              ...                        
9022    by the way, i love that y'all are so down to d...
9029    [top story] at #sxsw, apple schools the market...
9033    @mention yep! i can't believe they set up a po...
9048    @mention you bet man! kindle and apple for sur...
9064    @mention you should see the line here at #sxsw...
Name: tweet, Length: 638, dtype: object

In [22]:
df_binary['subject'].isna().sum()

357

In [23]:
# function to label new column based on the product category 

def label_brand(row):
    if row['subject'] == 'iPad':
        return 'Apple'
    elif row['subject'] == 'Apple':
        return 'Apple'
    elif row['subject'] == 'iPad or iPhone App':
        return 'Apple'
    elif row['subject'] == 'iPhone':
        return 'Apple'
    elif row['subject'] == 'Other Apple product or service':
        return 'Apple'
    elif row['subject'] == 'Google':
        return 'Google'
    elif row['subject'] == 'Other Google product or service':
        return 'Google'
    elif row['subject'] == 'Android App':
        return 'Google'
    elif row['subject'] == 'Android':
        return 'Google'
    elif row['subject'] == np.nan: 
        return np.nan

In [24]:
df_binary['company'] =  df_binary.apply(lambda row: label_brand(row), axis=1)

<ipython-input-24-21a226a385bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binary['company'] =  df_binary.apply(lambda row: label_brand(row), axis=1)


In [25]:
df_binary['company'].value_counts()

Apple     2337
Google     854
Name: company, dtype: int64

In [26]:
df_binary['company'].isna().sum()

357

In [27]:
df_binary.isna().sum()

tweet        0
subject    357
emotion      0
company    357
dtype: int64

In [28]:
len(df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains("Apple"))])

0

In [29]:
len(df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains("Google"))])

0

In [30]:
apple_words = ['apple','ipad','iphone','itunes']
google_words = ['google','android']

In [31]:
df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains('|'.join(apple_words))),'company'] = 'Apple'

C:\Users\User\anaconda3_1\envs\learn-env\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-31-eebd2101ff92>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains('|'.join(apple_words))),'company'] = 'Apple'


In [32]:
df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains('|'.join(google_words))),'company'] = 'Google'

<ipython-input-32-c57a5e0afc5b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binary.loc[(df_binary['subject'].isna()) & (df_binary['tweet'].str.contains('|'.join(google_words))),'company'] = 'Google'


In [33]:
df_binary['company'].value_counts()

Apple     2546
Google     991
Name: company, dtype: int64

In [34]:
df_binary.isna().sum()

tweet        0
subject    357
emotion      0
company     11
dtype: int64

In [60]:
#df_binary.loc[df_binary['company'].isna()]['tweet']

In [63]:
df_binary.dropna(subset=['company'],inplace=True)

<ipython-input-63-7638a198be87>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_binary.dropna(subset=['company'],inplace=True)


In [64]:
df_binary.isna().sum()

tweet        0
subject    346
emotion      0
company      0
dtype: int64

In [67]:
df_binary

,tweet,subject,emotion,company
0,.@wesley83 i have a 3g iphone. after 3 hrs twe...,iPhone,Negative emotion,Apple
1,@jessedee know about @fludapp ? awesome ipad/i...,iPad or iPhone App,Positive emotion,Apple
2,@swonderlin can not wait for #ipad 2 also. the...,iPad,Positive emotion,Apple
3,@sxsw i hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,Apple
4,@sxtxstate great stuff on fri #sxsw: marissa m...,Google,Positive emotion,Google
...,...,...,...,...
9077,@mention your pr guy just convinced me to swit...,iPhone,Positive emotion,Apple
9079,&quot;papyrus...sort of like the ipad&quot; - ...,iPad,Positive emotion,Apple
9080,diller says google tv &quot;might be run over ...,Other Google product or service,Negative emotion,Google
9085,i've always used camera+ for my iphone b/c it ...,iPad or iPhone App,Positive emotion,Apple


In [76]:
df_google = df_binary.loc[df_binary['company'] == 'Google']

In [77]:
X_google = df_google['tweet']
y_google = df_google['emotion']

In [78]:
df_apple = df_binary.loc[df_binary['company'] == 'Apple']

In [79]:
X_apple = df_apple['tweet']
y_apple = df_apple['emotion']

In [80]:
print(len(X_google) + len(X_apple))

3537


In [81]:
print(len(y_google) + len(y_apple))

3537
